# ABEL linac tracking example

By Carl A. Lindstrøm (University of Oslo), 8 Sep 2023

### Import ABEL framework

In [1]:
import sys
sys.path.append('../')
from abel import *
import numpy as np
import time


%load_ext autoreload
%autoreload 2

### Define the linac and beams

In [2]:
# define driver
driver = SourceBasic()
driver.charge = -2.7e10 * SI.e # [C]
driver.energy = 31.25e9 # [eV]
driver.rel_energy_spread = 0.01
driver.bunch_length = 50e-6 # [m]
driver.z_offset = 580e-6 # [m]
driver.emit_nx, driver.emit_ny = 80e-6, 80e-6 # [m rad]
driver.beta_x, driver.beta_y = 5, 5 #30e-3, 30e-3 # [m]
driver.num_particles = 100000
driver.jitter.x = 0 # [m] 2e-6
driver.jitter.y = 0 # [m] 100e-9
driver.jitter.t = 0 # 10e-15 # [m]
drive_beam = driver.track()  # Convert a Source object to a Beam object.  ######

# define stage
stage = StageQuasistatic2d()
#stage = StagePrtclTransWakeInstability()                      #######
stage.drive_beam = drive_beam                                 #######
stage.driver_source = driver
stage.nom_energy_gain = 31.9e9 # [eV]
stage.length = 11 # [m]
stage.plasma_density = 2e21 # [m^-3]
stage.ramp_beta_mag = 5
stage.radiation_reaction = False
stage.save_evolution = False

# define first stage (half length)
first_stage = StageQuasistatic2d()
#first_stage = StagePrtclTransWakeInstability()                      #######
first_stage.drive_beam = drive_beam                                 #######
first_stage.driver_source = driver
first_stage.nom_energy_gain = stage.nom_energy_gain/2
first_stage.length = stage.length/2
first_stage.plasma_density = stage.plasma_density
first_stage.ramp_beta_mag = stage.ramp_beta_mag
first_stage.radiation_reaction = stage.radiation_reaction
first_stage.save_evolution = stage.save_evolution

# define beam
source = SourceBasic()
source.charge = -1e10 * SI.e # [C]
source.energy = 5e9 # [eV]
#source.rel_energy_spread = 0.1
source.rel_energy_spread = 0.01
source.bunch_length = 28e-6 # [m]
source.z_offset = 0e-6 # [m]
source.emit_nx, source.emit_ny = 160e-6, 0.56e-6 # [m rad]
source.beta_x = stage.matched_beta_function(source.energy)
source.beta_y = source.beta_x
source.num_particles = 50000
source.jitter.t = driver.jitter.t
"""
main_gamma = energy2gamma(source.energy)
main_sigma_x = np.sqrt(source.emit_nx/main_gamma*source.beta_x)  # [m]
main_sigma_y = np.sqrt(source.emit_ny/main_gamma*source.beta_y)  # [m]
source.x_offset = main_sigma_x*0.1  # [m]
source.y_offset = main_sigma_y*0.1  # [m]
"""
# define interstage
interstage = InterstageBasic()
#interstage = InterstageElegant()
interstage.beta0 = lambda E: stage.matched_beta_function(E)
interstage.dipole_length = lambda E: 1 * np.sqrt(E/10e9) # [m(eV)]
interstage.dipole_field = 1.  # [T]
#interstage.dipole_field = lambda E: np.min([1.0, 100e9/E]) # [T]

# beam delivery system
bds = BeamDeliverySystemBasic()
bds.beta_x, bds.beta_y = 8e-3, 0.4e-3 # [m]
bds.bunch_length = 0.75 * bds.beta_y

# define linac
linac = Linac()
linac.source = source
linac.stage = stage
linac.first_stage = first_stage
linac.interstage = interstage
linac.num_stages = 16

### Run simulations

In [3]:
start = time.time()
linac.run('linac_energy_loss_first_stages', num_shots=1, overwrite=True, parallel=False);
end = time.time()

print((end - start)/60, ' min') # time in minutes

Tracking element 1 (SourceBasic, stage 0, s = 0.0 m, 5.0 GeV, -1.60 nC, 1.0% rms, 160.5/0.6 µm-rad)
Tracking element 2 (StageQuasistatic2d, stage 1, s = 5.5 m, 21.0 GeV, -1.60 nC, 3.1% rms, 160.0/0.6 µm-rad)
Tracking element 3 (InterstageBasic, stage 1, s = 12.4 m, 21.0 GeV, -1.60 nC, 3.1% rms, 159.9/0.6 µm-rad)
Tracking element 4 (StageQuasistatic2d, stage 2, s = 23.4 m, 52.8 GeV, -1.60 nC, 1.5% rms, 160.4/0.6 µm-rad)
Tracking element 5 (InterstageBasic, stage 2, s = 34.4 m, 52.8 GeV, -1.60 nC, 1.5% rms, 160.4/0.6 µm-rad)
Tracking element 6 (StageQuasistatic2d, stage 3, s = 45.4 m, 84.6 GeV, -1.60 nC, 0.8% rms, 160.0/0.6 µm-rad)
Tracking element 7 (InterstageBasic, stage 3, s = 59.4 m, 84.6 GeV, -1.60 nC, 0.8% rms, 160.1/0.6 µm-rad)
Tracking element 8 (StageQuasistatic2d, stage 4, s = 70.4 m, 116.5 GeV, -1.60 nC, 0.5% rms, 159.7/0.6 µm-rad)
Tracking element 9 (InterstageBasic, stage 4, s = 86.7 m, 116.5 GeV, -1.60 nC, 0.5% rms, 159.7/0.6 µm-rad)
Tracking element 10 (StageQuasistatic2d

### Plot the wakefield

In [4]:
# define driver
driver = SourceBasic()
driver.charge = -2.7e10 * SI.e # [C]
driver.energy = 31.25e9 # [eV]
driver.rel_energy_spread = 0.01
driver.bunch_length = 50e-6 # [m]
driver.z_offset = 580e-6 # [m]
driver.emit_nx, driver.emit_ny = 80e-6, 80e-6 # [m rad]
driver.beta_x, driver.beta_y = 5, 5 #30e-3, 30e-3 # [m]
driver.num_particles = 100000
driver.jitter.x = 0 # [m] 2e-6
driver.jitter.y = 0 # [m] 100e-9
driver.jitter.t = 0 # 10e-15 # [m]
drive_beam = driver.track()  # Convert a Source object to a Beam object.  ######

# define stage
stage = StageQuasistatic2d()
#stage = StagePrtclTransWakeInstability()                      #######
stage.drive_beam = drive_beam                                 #######
stage.driver_source = driver
stage.nom_energy_gain = 31.9e9 # [eV]
stage.length = 11 # [m]
stage.plasma_density = 2e21 # [m^-3]
stage.ramp_beta_mag = 5
stage.radiation_reaction = False
stage.save_evolution = False

end_beam = linac.get_beam(-1)
new_source = SourceBasic()
new_source.charge = end_beam.charge()
new_source.energy = end_beam.energy()
new_source.rel_energy_spread = end_beam.rel_energy_spread()
new_source.bunch_length = end_beam.bunch_length()
new_source.z_offset = end_beam.z_offset()
new_source.emit_nx, source.emit_ny = end_beam.norm_emittance_x(), end_beam.norm_emittance_y()
new_source.beta_x = end_beam.beta_x()
new_source.beta_y = end_beam.beta_y()
new_source.num_particles = len(end_beam.xs())
new_source.jitter.t = driver.jitter.t

In [5]:
new_linac = Linac()
new_linac.source = new_source
new_linac.stage = stage
#linac.first_stage = first_stage
#linac.interstage = interstage
new_linac.num_stages = 1

In [6]:
start = time.time()
new_linac.run('linac_Energy_loss_new_source', num_shots=1, overwrite=True, parallel=False);
end = time.time()

print((end - start)/60, ' min') # time in minutes

ZeroDivisionError: Weights sum to zero, can't be normalized